In [1]:
import pandas as pd
import random
from fuzzywuzzy import fuzz
from faker import Faker
from faker.providers import person
from faker.providers import address
from faker.providers import date_time
import datetime
import numpy as np

/var/mobile/Containers/Data/Application/0D417CDB-1B7A-4E2E-AE0D-A02D81A235A4/Documents/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [98]:
def pp(start, end, n=1):
    start_u = start.value//10**9
    end_u = end.value//10**9
    #return pd.DatetimeIndex((10**9*np.random.randint(start_u, end_u, n, dtype=np.int64)).view('M8[ns]')).to_pydatetime()
    return pd.DatetimeIndex((10**9*np.random.randint(start_u, end_u, n, dtype=np.int64)).view('M8[ns]')).normalize()[0]

In [99]:
def generate_record(fake):
    d = dict()
    
    d['gender'] = 'M' if random.randint(0,1) == 0 else 'F'
    d['title']= fake.prefix_male() if d['gender']=="M" else fake.prefix_female()
    d['first_name'] = fake.first_name_male() if d['gender']=='M' else fake.first_name_female()
    d['middle_name'] = fake.first_name() if random.randint(0,5) == 0 else ''
    
    d['middle_name']= d['middle_name'][0] if (d['middle_name']!='') and (random.randint(0,5)) == 0 else d['middle_name']
    d['last_name'] = fake.last_name()
    d['suffix']=fake.suffix() if random.randint(0,30) == 0 else ''
    d['country']=fake.country()
    d['city']=fake.city()
    #note that this library has 1970 limit, for earlier dates you need a custom formula and it is trickier
    #d['dob'] = fake.date_between_dates(date_start=datetime.date(1970,1,1), date_end=datetime.date.today())
    start = pd.to_datetime('1940-1-1')
    end = pd.to_datetime('today')
    d['dob']=pp(start,end)
    
    d['address']= fake.street_address()
    d['member_id']=str(fake.ean(length=13))
    d['full_address']=d['address']+', '+ d['city']+', '+ d['country']
    
    if random.randint(0,9)>0:
        d['full_name']= d['first_name']+ ' ' + d['middle_name']+ ' '+d['last_name']+' '+d['suffix']
    else:
        d['full_name']= d['last_name']+ ', '+ d['first_name']+ ' ' + d['middle_name']+(', ' if d['suffix'] != '' else '') +d['suffix']
    
    d['full_name']= " ".join(d['full_name'].split())
    return d

fake = Faker()
fake.add_provider(person)
fake.add_provider(address)
fake.add_provider(date_time)
output = pd.DataFrame()
rec=None

for i in range(200):
    rec=generate_record(fake)
    output = output.append(rec, ignore_index=True)  
output

,address,city,country,dob,first_name,full_address,full_name,gender,last_name,member_id,middle_name,suffix,title
0,408 Perez Station Suite 465,Archerfurt,Swaziland,1949-04-26,John,"408 Perez Station Suite 465, Archerfurt, Swazi...",John Pace,M,Pace,4481319980577,,,Mr.
1,82917 Linda Mountains Suite 365,Carpenterfort,Singapore,1969-01-02,Ashley,"82917 Linda Mountains Suite 365, Carpenterfort...",Ashley Smith,F,Smith,3455455677746,,,Mrs.
2,2468 Carlson Club,Joseside,Seychelles,2010-05-16,Timothy,"2468 Carlson Club, Joseside, Seychelles","Harris, Timothy",M,Harris,1769869069908,,,Mr.
3,828 Love Plains,Grantbury,Saudi Arabia,1975-05-12,Leslie,"828 Love Plains, Grantbury, Saudi Arabia","Davis, Leslie Eric",F,Davis,2559556354245,Eric,,Dr.
4,508 Bird Roads,New Adriana,Suriname,1957-12-14,Elizabeth,"508 Bird Roads, New Adriana, Suriname",Elizabeth M Hensley,F,Hensley,1179459125578,M,,Ms.
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,7306 Hanson Keys,Lake Kathy,Philippines,1976-10-16,Dana,"7306 Hanson Keys, Lake Kathy, Philippines",Dana James,F,James,1277018563103,,,Miss
196,76897 Perry Extensions Apt. 028,Lake Jenna,Georgia,1970-05-20,Rebecca,"76897 Perry Extensions Apt. 028, Lake Jenna, G...",Rebecca Vazquez,F,Vazquez,0273174137838,,,Mrs.
197,082 Bridges Junctions Suite 307,West Melissastad,Portugal,1947-06-21,Franklin,"082 Bridges Junctions Suite 307, West Melissas...",Franklin Leah Peters,M,Peters,7343040004144,Leah,,Mr.
198,363 Ramirez Bridge Apt. 551,Lake David,Guyana,1970-10-30,Desiree,"363 Ramirez Bridge Apt. 551, Lake David, Guyana",Desiree Mclean,F,Mclean,9854714284116,,,Dr.


In [104]:
Str1 = "Apple Inc."
Str2 = "apple Inc"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
print(Ratio)

95


In [105]:
Str1 = "Los Angeles Lakers"
Str2 = "L.A. Lakers"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
Token_Sort_Ratio = fuzz.token_sort_ratio(Str1.lower(),Str2.lower())
print(Ratio)
print(Partial_Ratio)
print(Token_Sort_Ratio)

62
73
64


In [106]:
Str1 = "united states v. nixon"
Str2 = "Nixon v. United States"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
Token_Sort_Ratio = fuzz.token_sort_ratio(Str1,Str2)
print(Ratio)
print(Partial_Ratio)
print(Token_Sort_Ratio)

59
74
100
